In [15]:
import polars as pl
import os
from datetime import datetime, timezone, timedelta
import plotly.express as px
import numpy as np
import csv

from sites_deloyment_times import deployment_times, datetime_format

DATA_DIRECTORY = os.environ.get("DATA_DIRECTORY")


In [16]:
sites_meta = pl.read_csv("sites.csv", separator=";").with_columns(pl.exclude(pl.Utf8).cast(str))

df_1_min = pl.scan_parquet(os.path.join(DATA_DIRECTORY, "processed", "pipeline", "flagged_1_min_acropolis.parquet"))
df_1_h = pl.scan_parquet(os.path.join(DATA_DIRECTORY, "processed", "pipeline", "flagged_1_h_acropolis.parquet"))

In [ ]:
print(deployment_times["TUMR"])
sites_meta.filter(pl.col("site") == "TUMR")

In [32]:
# Utility

def extract_site_data(df, deployment_times: dict, site_name:str):
    extracted_dates = []
    
    for sensor in deployment_times[site_name]["sensors"]:
        
        id = sensor["id"]
        start_time = datetime.strptime(sensor["start_time"], datetime_format)
        end_time = datetime.strptime(sensor["end_time"], datetime_format)       
        
        df_temp = df.filter(pl.col("system_id") == id)  \
            .filter(pl.col("creation_timestamp") \
            .is_between(start_time, end_time)) \
            .collect()
        
        extracted_dates.append(df_temp)
        
    return pl.concat(extracted_dates)

def plot_1_h_product(df_1_h, wind_direction_range=[]):
    
    df_plot = df_1_h.filter(pl.col("Flag") == 'O')
    
    if wind_direction_range != []:
        df_plot = df_plot.filter((pl.col("wxt532_direction_avg") > min(wind_direction_range)) & (pl.col("wxt532_direction_avg") < max(wind_direction_range)))
    
    fig = px.line(df_plot, 
            x="creation_timestamp", 
            y="gmp343_corrected", 
            markers=True)
    fig.update_traces(connectgaps=False)
    fig.update_layout(
        yaxis_title='CO2 Concentration (ppm)',
        xaxis_title='',
        title='',
    )
    fig.show()
    
    fig = px.line(df_plot, 
            x="creation_timestamp", 
            y="wxt532_direction_avg", 
            markers=True)
    fig.update_traces(connectgaps=False)
    fig.show()
    
    
def plot_1_min_product(df_1_min):
       
    df_plot = df_1_min.with_columns(
    pl.when(pl.col("OriginalFlag") == 185)
    .then(pl.col("gmp343_corrected"))
    .otherwise(np.nan) 
    .alias("co2_185")
    ) \
    .with_columns(
    pl.when(pl.col("OriginalFlag") == 0)
    .then(pl.col("gmp343_corrected"))
    .otherwise(np.nan)
    .alias("co2_0")
    )
    
    fig = px.scatter(df_plot, 
            x="creation_timestamp", 
            y="wxt532_direction_avg", 
            )
    fig.update_traces(connectgaps=False)
    fig.update_layout(
        yaxis_title='Wind Direction (°)',
        xaxis_title='',
        title='',
    )
    fig.update_yaxes(range=[0, 360])
    fig.show()
    
    fig = px.scatter(df_plot, 
            x="creation_timestamp", 
            y=["co2_0", "co2_185"], 
            )
    fig.update_traces(connectgaps=False)
    fig.update_layout(
        yaxis_title='CO2 Concentration (ppm)',
        xaxis_title='',
        title='',
    )
    fig.update_yaxes(range=[400, 1000])
    fig.show()
    
def calculate_decimal_year(date) -> float:
    year = date.year 
    today = (date - datetime(year, 1, 1, 0, 0, 0).replace(tzinfo=timezone.utc)).total_seconds()
    seconds_total_year = (datetime(year, 1, 1, 0, 0, 0).replace(tzinfo=timezone.utc) - datetime(year-1, 1, 1, 0, 0, 0).replace(tzinfo=timezone.utc)).total_seconds()

    x=  ((today / seconds_total_year) + year)
    return float("{:.6f}".format(x))

# 1 min data (Level 1)

In [ ]:
# Test export
start_date = datetime(2024, 12, 8, 0, 0, 0).replace(tzinfo=timezone.utc)
end_date = datetime(2024, 12, 13, 23, 59, 59).replace(tzinfo=timezone.utc)

df_temp = df_1_min.filter(pl.col("creation_timestamp").is_between(start_date, end_date))
df_site = extract_site_data(df=df_temp, deployment_times = deployment_times, site_name="MAIR")
plot_1_min_product(df_site)

In [ ]:
df_1_min.sort("creation_timestamp").head(1).collect()

In [ ]:
for site in deployment_times.keys():
    print(site)
    
    df_temp = extract_site_data(df=df_1_min, deployment_times = deployment_times, site_name=site) \
        .fill_null('') \
        .rename({"gmp343_corrected": "co2", "h2o_v%": "h2o", "enclosure_bme280_pressure":"pressure", "gmp343_temperature":"sensor_temperature", "wxt532_speed_avg": "ws", "wxt532_direction_avg":"wd"}) \
        .with_columns((pl.col("creation_timestamp") + timedelta(seconds=30))) \
        .with_columns(
            pl.col("co2").round(2),
            pl.col("h2o").round(2),
            pl.col("pressure").round(2),
            pl.col("sensor_temperature").round(2),
            pl.col("ws").round(2),
            pl.col("wd").round(2)) \
        .with_columns(
            (pl.col("creation_timestamp").dt.year()).alias("Year"),
            (pl.col("creation_timestamp").dt.month()).alias("Month"),
            (pl.col("creation_timestamp").dt.day()).alias("Day"),
            (pl.col("creation_timestamp").dt.hour()).alias("Hour"),
            (pl.col("creation_timestamp").dt.minute()).alias("Minute"),
            (pl.col("creation_timestamp").dt.second()).alias("Second"),
            (pl.col('creation_timestamp').dt.to_string("%Y-%m-%d %H:%M:%S")).alias("#Datetime")) \
        .with_columns(pl.struct(['creation_timestamp']) \
        .map_elements(lambda x: calculate_decimal_year(x['creation_timestamp']), return_dtype=pl.Float64) \
        .alias("DecimalDate")) \
        .select(["#Datetime", "Year", "Month", "Day", "Hour", "Minute", "Second", "DecimalDate", "co2", "h2o", "pressure", "sensor_temperature", "ws", "wd", "Flag", "OriginalFlag"]) \
        .with_columns(pl.exclude(pl.Utf8).cast(str)) \
        .fill_null('')
        
    # construct icos cities portal head

    file_name = f"{site}_munich_acropolis_L1_1min.csv"
    file_lines = len(df_temp) + 46
    data_level = 1
    site_short_name = site[:4]
    site_long_name = sites_meta.filter(pl.col("site") == site[:4]).select("site_name").item()
    latitude = sites_meta.filter(pl.col("site") == site[:4]).select("latitude").item()
    longitude = sites_meta.filter(pl.col("site") == site[:4]).select("longitude").item()
    altitude = sites_meta.filter(pl.col("site") == site[:4]).select("elevation").item()
    sampling_height = sites_meta.filter(pl.col("site") == site[:4]).select("height_of_building").item()
    start_date = df_temp.select("#Datetime").row(0)[0]
    stop_date = df_temp.select("#Datetime").row(-1)[0]

    with open(os.path.join(DATA_DIRECTORY,"processed", "icos_cities_portal_processing", "level_1",file_name), 'w', newline='') as file:
        writer = csv.writer(file, delimiter=';', lineterminator='\n')
        field = ["#Datetime", "Year", "Month", "Day", "Hour", "Minute", "Second", "DecimalDate", "co2", "h2o", "pressure", "sensor_temperature", "ws", "wd", "Flag", "OriginalFlag"]
    
        writer.writerow(["# TITLE: co2 - continuous time series from low and mid cost sensors"])
        writer.writerow([f"# FILE NAME: {file_name}"])
        writer.writerow(["# DATA FORMAT: see the last line of this header for column description"])
        writer.writerow([f'# TOTAL LINES: {file_lines}'])
        writer.writerow(['# HEADER LINES: 46'])
        writer.writerow(['# PROJECT: ICOS CITIES'])
        writer.writerow([f'# DATA VERSION: L{data_level}'])
        writer.writerow([f'# STATION CODE: {site_short_name}'])
        writer.writerow([f'# STATION NAME: {site_long_name} ({site_short_name})'])
        writer.writerow(['# OBSERVATION CATEGORY: Air sampling observation at a stationary platform'])
        writer.writerow(['# COUNTRY/TERRITORY: DE'])
        writer.writerow([r'# RESPONSIBLE INSTITUTE: TUM, Technial University Munich'])
        writer.writerow(['# CONTRIBUTOR:  Patrick Aigner, Jia Chen, Klaus Kürzinger'])
        writer.writerow(['# CONTACT POINT: Patrick Aigner <patrick.aigner@tum.de>, Jia Chen <jia.chen@tum.de>'])
        writer.writerow(["# FUNDING: European Union's Horizon 2020 Research and Innovation Programme, Grant Agreement No. 101037319"])
        writer.writerow([f'# LATITUDE: {latitude}'])
        writer.writerow([f'# LONGITUDE: {longitude}'])
        writer.writerow([f'# ALTITUDE: {altitude} m asl'])
        
        if site[:4] == "BLUT":
            writer.writerow([f'# SAMPLING HEIGHTS: {site[-2:]} m agl'])
        else:
            writer.writerow([f'# SAMPLING HEIGHTS: {sampling_height} m agl'])
        
        writer.writerow(['# PARAMETER: co2'])
        writer.writerow([f'# COVERING PERIOD: {start_date} - {stop_date}'])
        writer.writerow(['# TIME INTERVAL: hourly'])
        writer.writerow(['# MEASUREMENT UNIT: µmol/mol'])
        writer.writerow(['# MEASUREMENT METHOD: NDIR'])
        writer.writerow(['# INSTRUMENT: Vaisala GMP343'])
        writer.writerow(['# SAMPLING TYPE: continuous'])
        writer.writerow(['# TIME ZONE: Central European Time (UTC+1), Central European Summer Time (UTC+2)'])
        writer.writerow(['# MEASUREMENT SCALE: WMO-CO2-X2019'])
        writer.writerow(['# DATA POLICY: ICOS CITIES DATA is licensed under a Creative Commons Attribution 4.0 international licence (http://creativecommons.org/licenses/by/4.0/.The ICOS CITIES data licence is described at https://data.icos-cities.eu/licence.'])
        writer.writerow(['# COMMENT:'])
        writer.writerow(['#'])
        writer.writerow(['#   - Times are UTC+0'])
        writer.writerow(['#   - Time-averaged values are reported at the middle of the averaging interval.'])
        writer.writerow(['#   - co2: dry mole air fraction (µmol/mol)'])
        writer.writerow(['#   - h20: absolute humidity in vol%'])
        writer.writerow(['#   - pressure: ambient pressure of outdoor enclosure in hPa'])
        writer.writerow(['#   - sensor_temperature: measurement chamber temperature in °C'])
        writer.writerow(['#   - ws: wind speed in m/s'])
        writer.writerow(['#   - wd: wind direction in degrees'])
        writer.writerow(['#   - OriginalFlag: based on flags from the EBAS system'])
        writer.writerow(["#   - Flag 'O' = data correct after manual quality control"])
        writer.writerow(["#   - Flag 'K' = data incorrect after manual quality control"])
        writer.writerow(['#   - In case of gaps between instruments, the timeseries are filled with empty string'])
        writer.writerow(['#   - Release notes: '])
        writer.writerow(['#'])
        writer.writerow(field)
        
        for row in df_temp.iter_rows():
            writer.writerow([''.join(item) for item in row])


# 1 hour data (Level 2)

In [ ]:
# Test export
df_site = extract_site_data(df=df_1_h, deployment_times = deployment_times, site_name="BALR")
plot_1_h_product(df_site)

# Manual Checks and Correction

In [29]:
# function

def flag_and_plot(df, system_id: int, site_id:str, start_date, end_date):

    df_temp = df_1_min.filter(pl.col("creation_timestamp").is_between(start_date, end_date))
    df_site = extract_site_data(df=df_temp, deployment_times = deployment_times, site_name=site_id)
    plot_1_min_product(df_site)
    
    return df.with_columns(
        pl.when(
            pl.col("creation_timestamp").is_between(start_date, end_date) &
            (pl.col("system_id") == system_id)
        )
        .then(pl.lit("K")) 
        .otherwise(pl.col("Flag"))  # Keep original value for others
        .alias("Flag")
    )

In [30]:
df_manual_check = df_1_h

In [ ]:
# Flags for 1 minute

# 'H' Potentially locally contaminated by hampel filter (auto)
# 'W' Possible local contamination indicated by wind direction or velocity (auto)
# 'C' Potentially locally contaminated (manual quality control)
# 'T' Sensor temperature instability 
# 'I' Sub-optimal inlet placement

In [33]:
# Manually update flags for PASR on August 13.
# Readings might be related to a summer party on the roof-top

start_date = datetime(2024, 8, 13, 17, 0, 0).replace(tzinfo=timezone.utc)
end_date = datetime(2024, 8, 13, 19, 59, 59).replace(tzinfo=timezone.utc)

df_manual_check = flag_and_plot(df=df_manual_check, system_id=18, site_id="PASR", start_date=start_date, end_date=end_date)

In [35]:
# Manually update flags for BALR 

# Remove period
start_date = datetime(2024, 11, 2, 1, 30, 0).replace(tzinfo=timezone.utc)
end_date = datetime(2024, 11, 2, 4, 30, 0).replace(tzinfo=timezone.utc)

df_manual_check = flag_and_plot(df=df_manual_check, system_id=19, site_id="BALR", start_date=start_date, end_date=end_date)

# Remove period
start_date = datetime(2024, 11, 2, 6, 0, 0).replace(tzinfo=timezone.utc)
end_date = datetime(2024, 11, 2, 7, 30, 0).replace(tzinfo=timezone.utc)

df_manual_check = flag_and_plot(df=df_manual_check, system_id=19, site_id="BALR", start_date=start_date, end_date=end_date)

# Remove period
start_date = datetime(2024, 11, 21, 20, 50, 0).replace(tzinfo=timezone.utc)
end_date = datetime(2024, 11, 22, 2, 20, 0).replace(tzinfo=timezone.utc)

df_manual_check = flag_and_plot(df=df_manual_check, system_id=19, site_id="BALR", start_date=start_date, end_date=end_date)

# Remove period
start_date = datetime(2024, 11, 28, 6, 30, 0).replace(tzinfo=timezone.utc)
end_date = datetime(2024, 11, 28, 8, 30, 0).replace(tzinfo=timezone.utc)

df_manual_check = flag_and_plot(df=df_manual_check, system_id=19, site_id="BALR", start_date=start_date, end_date=end_date)

# Remove period
start_date = datetime(2024, 11, 13, 9, 0, 0).replace(tzinfo=timezone.utc)
end_date = datetime(2024, 11, 13, 20, 0, 0).replace(tzinfo=timezone.utc)

df_manual_check = flag_and_plot(df=df_manual_check, system_id=19, site_id="BALR", start_date=start_date, end_date=end_date)

# Remove period
start_date = datetime(2024, 12, 4, 0, 0, 0).replace(tzinfo=timezone.utc)
end_date = datetime(2024, 12, 4, 2, 0, 0).replace(tzinfo=timezone.utc)

df_manual_check = flag_and_plot(df=df_manual_check, system_id=19, site_id="BALR", start_date=start_date, end_date=end_date)

# Remove period
start_date = datetime(2024, 12, 8, 15, 0, 0).replace(tzinfo=timezone.utc)
end_date = datetime(2024, 12, 8, 17, 0, 0).replace(tzinfo=timezone.utc)

df_manual_check = flag_and_plot(df=df_manual_check, system_id=19, site_id="BALR", start_date=start_date, end_date=end_date)

In [ ]:
# Manually update flags for NPLR 

# Remove period
start_date = datetime(2024, 12, 10, 14, 30, 0).replace(tzinfo=timezone.utc)
end_date = datetime(2024, 12, 10, 17, 0, 0).replace(tzinfo=timezone.utc)

df_manual_check = flag_and_plot(df=df_manual_check, system_id=9, site_id="NPLR", start_date=start_date, end_date=end_date)

# Remove period
start_date = datetime(2024, 12, 10, 8, 30, 0).replace(tzinfo=timezone.utc)
end_date = datetime(2024, 12, 11, 0, 30, 0).replace(tzinfo=timezone.utc)

df_manual_check = flag_and_plot(df=df_manual_check, system_id=9, site_id="NPLR", start_date=start_date, end_date=end_date)

In [ ]:
# Manually update flags for BOGR

# Remove period
start_date = datetime(2024, 10, 4, 16, 0, 0).replace(tzinfo=timezone.utc)
end_date = datetime(2024, 10, 5, 1, 0, 0).replace(tzinfo=timezone.utc)

df_manual_check = flag_and_plot(df=df_manual_check, system_id=17, site_id="BOGR", start_date=start_date, end_date=end_date)

In [37]:
# Manually update flags for MAIR

# Remove period
start_date = datetime(2024, 3, 7, 15, 0, 0).replace(tzinfo=timezone.utc)
end_date = datetime(2024, 3, 8, 5, 8, 0).replace(tzinfo=timezone.utc)

df_manual_check = flag_and_plot(df=df_manual_check, system_id=16, site_id="MAIR", start_date=start_date, end_date=end_date)


# Remove period
start_date = datetime(2024, 3, 8, 19, 30, 0).replace(tzinfo=timezone.utc)
end_date = datetime(2024, 3, 9, 1, 20, 0).replace(tzinfo=timezone.utc)

df_manual_check = flag_and_plot(df=df_manual_check, system_id=16, site_id="MAIR", start_date=start_date, end_date=end_date)


# Remove period
start_date = datetime(2024, 4, 22, 6, 10, 0).replace(tzinfo=timezone.utc)
end_date = datetime(2024, 4, 22, 8, 10, 0).replace(tzinfo=timezone.utc)

df_manual_check = flag_and_plot(df=df_manual_check, system_id=16, site_id="MAIR", start_date=start_date, end_date=end_date)


# Remove period
start_date = datetime(2024, 4, 22, 16, 20, 0).replace(tzinfo=timezone.utc)
end_date = datetime(2024, 4, 23, 11, 0, 0).replace(tzinfo=timezone.utc)

df_manual_check = flag_and_plot(df=df_manual_check, system_id=16, site_id="MAIR", start_date=start_date, end_date=end_date)


# Remove period
start_date = datetime(2024, 12, 8, 18, 0, 0).replace(tzinfo=timezone.utc)
end_date = datetime(2024, 12, 11, 11, 0, 0).replace(tzinfo=timezone.utc)

df_manual_check = flag_and_plot(df=df_manual_check, system_id=16, site_id="MAIR", start_date=start_date, end_date=end_date)

In [38]:
# Manually update flags for TUMR

# Remove period
start_date = datetime(2024, 10, 14, 7, 22, 0).replace(tzinfo=timezone.utc)
end_date = datetime(2024, 10, 14, 12, 53, 0).replace(tzinfo=timezone.utc)

df_manual_check = flag_and_plot(df=df_manual_check, system_id=6, site_id="TUMR", start_date=start_date, end_date=end_date)

# Export

In [ ]:
for site in deployment_times.keys():
    print(site)
    
    df_temp = extract_site_data(df=df_1_h, deployment_times = deployment_times, site_name=site) \
        .fill_null('') \
        .rename({"gmp343_corrected": "co2", "h2o_v%": "h2o", "enclosure_bme280_pressure":"pressure", "gmp343_temperature":"sensor_temperature", "wxt532_speed_avg": "ws", "wxt532_direction_avg":"wd"}) \
        .with_columns(
            pl.col("co2").round(2),
            pl.col("h2o").round(2),
            pl.col("pressure").round(2),
            pl.col("sensor_temperature").round(2),
            pl.col("ws").round(2),
            pl.col("wd").round(2),
            pl.col("Stdev").round(2)) \
        .with_columns(
            (pl.col("creation_timestamp").dt.year()).alias("Year"),
            (pl.col("creation_timestamp").dt.month()).alias("Month"),
            (pl.col("creation_timestamp").dt.day()).alias("Day"),
            (pl.col("creation_timestamp").dt.hour()).alias("Hour"),
            (pl.col("creation_timestamp").dt.minute()).alias("Minute"),
            (pl.col("creation_timestamp").dt.second()).alias("Second"),
            (pl.col('creation_timestamp').dt.to_string("%Y-%m-%d %H:%M:%S")).alias("#Datetime")) \
        .with_columns(pl.struct(['creation_timestamp']) \
        .map_elements(lambda x: calculate_decimal_year(x['creation_timestamp']), return_dtype=pl.Float64) \
        .alias("DecimalDate")) \
        .select(["#Datetime", "Year", "Month", "Day", "Hour", "Minute", "Second", "DecimalDate", "co2", "h2o", "pressure", "sensor_temperature", "ws", "wd", "NbPoints", "Stdev", "Flag", "OriginalFlag"]) \
        .with_columns(pl.exclude(pl.Utf8).cast(str)) \
        .fill_null('')
        
    # construct icos cities portal head

    file_name = f"{site}_munich_acropolis_L2_1h.csv"
    file_lines = len(df_temp) + 46
    data_level = 2
    site_short_name = site[:4]
    site_long_name = sites_meta.filter(pl.col("site") == site[:4]).select("site_name").item()
    latitude = sites_meta.filter(pl.col("site") == site[:4]).select("latitude").item()
    longitude = sites_meta.filter(pl.col("site") == site[:4]).select("longitude").item()
    altitude = sites_meta.filter(pl.col("site") == site[:4]).select("elevation").item()
    sampling_height = sites_meta.filter(pl.col("site") == site[:4]).select("height_of_building").item()
    start_date = df_temp.select("#Datetime").row(0)[0]
    stop_date = df_temp.select("#Datetime").row(-1)[0]

    with open(os.path.join(DATA_DIRECTORY,"processed", "icos_cities_portal_processing", "level_2",file_name), 'w', newline='') as file:
        writer = csv.writer(file, delimiter=';', lineterminator='\n')
        field = ["#Datetime", "Year", "Month", "Day", "Hour", "Minute", "Second", "DecimalDate", "co2", "h2o", "pressure", "sensor_temperature", "ws", "wd", "NbPoints", "Stdev", "Flag", "OriginalFlag"]
    
        writer.writerow(["# TITLE: co2 - continuous time series from low and mid cost sensors"])
        writer.writerow([f"# FILE NAME: {file_name}"])
        writer.writerow(["# DATA FORMAT: see the last line of this header for column description"])
        writer.writerow([f'# TOTAL LINES: {file_lines}'])
        writer.writerow(['# HEADER LINES: 46'])
        writer.writerow(['# PROJECT: ICOS CITIES'])
        writer.writerow([f'# DATA VERSION: L{data_level}'])
        writer.writerow([f'# STATION CODE: {site_short_name}'])
        writer.writerow([f'# STATION NAME: {site_long_name} ({site_short_name})'])
        writer.writerow(['# OBSERVATION CATEGORY: Air sampling observation at a stationary platform'])
        writer.writerow(['# COUNTRY/TERRITORY: DE'])
        writer.writerow([r'# RESPONSIBLE INSTITUTE: TUM, Technial University Munich'])
        writer.writerow(['# CONTRIBUTOR:  Patrick Aigner, Jia Chen, Klaus Kürzinger'])
        writer.writerow(['# CONTACT POINT: Patrick Aigner <patrick.aigner@tum.de>, Jia Chen <jia.chen@tum.de>'])
        writer.writerow(["# FUNDING: European Union's Horizon 2020 Research and Innovation Programme, Grant Agreement No. 101037319"])
        writer.writerow([f'# LATITUDE: {latitude}'])
        writer.writerow([f'# LONGITUDE: {longitude}'])
        writer.writerow([f'# ALTITUDE: {altitude} m asl'])
        
        if site[:4] == "BLUT":
            writer.writerow([f'# SAMPLING HEIGHTS: {site[-2:]} m agl'])
        else:
            writer.writerow([f'# SAMPLING HEIGHTS: {sampling_height} m agl'])
        
        writer.writerow(['# PARAMETER: co2'])
        writer.writerow([f'# COVERING PERIOD: {start_date} - {stop_date}'])
        writer.writerow(['# TIME INTERVAL: hourly'])
        writer.writerow(['# MEASUREMENT UNIT: µmol/mol'])
        writer.writerow(['# MEASUREMENT METHOD: NDIR'])
        writer.writerow(['# INSTRUMENT: Vaisala GMP343'])
        writer.writerow(['# SAMPLING TYPE: continuous'])
        writer.writerow(['# TIME ZONE: Central European Time (UTC+1), Central European Summer Time (UTC+2)'])
        writer.writerow(['# MEASUREMENT SCALE: WMO-CO2-X2019'])
        writer.writerow(['# DATA POLICY: ICOS CITIES DATA is licensed under a Creative Commons Attribution 4.0 international licence (http://creativecommons.org/licenses/by/4.0/.The ICOS CITIES data licence is described at https://data.icos-cities.eu/licence.'])
        writer.writerow(['# COMMENT:'])
        writer.writerow(['#'])
        writer.writerow(['#   - Times are UTC+0'])
        writer.writerow(['#   - Time-averaged values are reported at the middle of the averaging interval.'])
        writer.writerow(['#   - co2: dry mole air fraction (µmol/mol)'])
        writer.writerow(['#   - h20: absolute humdity in vol%'])
        writer.writerow(['#   - pressure: ambient pressure of outdoor enclosure in hPa'])
        writer.writerow(['#   - sensor_temperature: measurement chamber temperature in °C'])
        writer.writerow(['#   - ws: wind speed in m/s'])
        writer.writerow(['#   - wd: wind direction in degrees'])
        writer.writerow(['#   - OriginalFlag: based on flags from the EBAS system'])
        writer.writerow(["#   - Flag 'O' = data correct after manual quality control"])
        writer.writerow(["#   - Flag 'K' = data incorrect after manual quality control"])
        writer.writerow(['#   - In case of gaps between instruments, the timeseries are filled with empty string'])
        writer.writerow(['#   - Release notes: '])
        writer.writerow(['#'])
        writer.writerow(field)
        
        for row in df_temp.iter_rows():
            writer.writerow([''.join(item) for item in row])
